In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import pickle
from text_processing import preprocess_text
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

df = pd.read_excel('../data_en.xlsx', nrows=7600)

In [2]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Tokenize and preprocess the text
texts = df['text'].astype(str).apply(preprocess_text).values

In [3]:
tokenizer = Tokenizer() 
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
data = pad_sequences(sequences)
print(data.shape)

with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


(7590, 50)


In [4]:
labels = df['id'].values
encoder = LabelEncoder()
labels = encoder.fit_transform(labels)
labels = to_categorical(labels)

with open('encoder.pickle', 'wb') as handle:
    pickle.dump(encoder, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [5]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.3, random_state=42, shuffle=True)

In [6]:
embedding_dim = 90
LSTM1 = 80
LSTM2 = 90
optimalizer = 'adam'
batch_size = 2
epochs = 8

In [7]:
VOCAB_SIZE = len(tokenizer.word_index) + 1
EMBEDDING_DIM = embedding_dim
MAX_LEN = len(data[0])
print(MAX_LEN)
OUTPUT_CLASSES = len(y_train[0])
print(OUTPUT_CLASSES)

50
33


Model

In [8]:
model = Sequential()
model.add(Embedding(VOCAB_SIZE, EMBEDDING_DIM, input_length=MAX_LEN))
model.add(LSTM(LSTM1, return_sequences=True))  # Změna na True pro více vrstev LSTM
model.add(Dropout(0.5))  # Přidání Dropout vrstvy
model.add(LSTM(LSTM2))  # Další vrstva LSTM
model.add(Dropout(0.5))  # Přidání Dropout vrstvy
model.add(Dense(OUTPUT_CLASSES, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=optimalizer,
              metrics=['accuracy'])


In [9]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,  # Zvýšení počtu epoch
          validation_data=(X_test, y_test))

Epoch 1/8
2657/2657 [==============================] - 75s 27ms/step - loss: 2.4941 - accuracy: 0.2710 - val_loss: 1.3570 - val_accuracy: 0.5832
Epoch 2/8
2657/2657 [==============================] - 77s 29ms/step - loss: 0.9243 - accuracy: 0.7267 - val_loss: 0.6440 - val_accuracy: 0.8173
Epoch 3/8
2657/2657 [==============================] - 79s 30ms/step - loss: 0.5010 - accuracy: 0.8526 - val_loss: 0.5258 - val_accuracy: 0.8507
Epoch 4/8
2657/2657 [==============================] - 83s 31ms/step - loss: 0.3661 - accuracy: 0.8931 - val_loss: 0.4916 - val_accuracy: 0.8608
Epoch 5/8
2657/2657 [==============================] - 104s 39ms/step - loss: 0.2705 - accuracy: 0.9183 - val_loss: 0.4633 - val_accuracy: 0.8827
Epoch 6/8
2657/2657 [==============================] - 84s 31ms/step - loss: 0.2018 - accuracy: 0.9401 - val_loss: 0.4564 - val_accuracy: 0.8845
Epoch 7/8
2657/2657 [==============================] - 89s 34ms/step - loss: 0.1762 - accuracy: 0.9430 - val_loss: 0.4124 - val_a

In [10]:
model.save('text_model_en')

INFO:tensorflow:Assets written to: text_model_en\assets


INFO:tensorflow:Assets written to: text_model_en\assets


In [29]:
from keras import models
from keras.models import load_model
import pickle
from text_processing import preprocess_text
model = load_model('../models/text_models/text_model_en')

In [35]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from text_processing import preprocess_text
class_names = ["lukostřelba", "baseball", "basketbal", "kulečník", "bmx", "bowling", "box", "jízda na býku", "roztleskávání", "curling", "šerm", "krasobruslení", "fotbal", "závody formule 1", "golf", "skok do výšky", "hokej", "dostihy", "hydroplánové závody", "judo", "motocyklové závody", "pole dance", "rugby", "skoky na lyžích", "snowboarding", "rychlobruslení", "surfování", "plavání", "stolní tenis", "tenis", "dráhové kolo", "volejbal", "vzpírání"]
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
with open('encoder.pickle', 'rb') as handle:
    encoder = pickle.load(handle)
    string = preprocess_text('fighting sport where figters box in the ring with gloves and one referee')
sequences = tokenizer.texts_to_sequences([string])
data = pad_sequences(sequences, maxlen=50)
predictions = model.predict(data)
predicted_labels = predictions.argmax(axis=-1)
predicted_labels = encoder.inverse_transform(predicted_labels)
predicted_confidence = predictions.max(axis=-1)
print(class_names[predicted_labels[0]])
print(f"jistota: {predicted_confidence[0]*100} %")

1/1 [==============================] - 0s 30ms/step
judo
jistota: 99.83446598052979 %
